In [3]:
import os
print("cwd =", os.getcwd())
print("files here:", os.listdir(".")[:50])

cwd = /home/david/thesis/figures
files here: ['Fig04_AGN_var_vs_index.png', 'Fig01_GRB_T90_hist.png', 'Untitled.ipynb', 'Fig05_CR_proton_flux_time_3p29_3p64GV.png', 'Fig02_GRB_fluence_vs_T90.png', '.ipynb_checkpoints', 'Fig05_unified_variability.png', 'Fig03_AGN_photon_index_BLL_FSRQ.png', 'Untitled1.ipynb']


In [6]:
import glob
matches = glob.glob("/home/david/**/*.csv", recursive=True)
print("num csv:", len(matches))
print("first 30:", matches[:30])

num csv: 141
first 30: ['/home/david/thesis/outputs/tables/agn_summary_stats.csv', '/home/david/thesis/outputs/tables/grb_summary_stats.csv', '/home/david/thesis/outputs/tables/cr_proton_modulation_summary.csv', '/home/david/thesis/data/raw/grb/fermigbrst.csv', '/home/david/thesis/data/raw/ams/AMS_timedependent_datasets/e+_AMS_PRL2023_daily.csv', '/home/david/thesis/data/raw/ams/AMS_timedependent_datasets/N-He_AMS_PRL2025_BR.csv', '/home/david/thesis/data/raw/ams/AMS_timedependent_datasets/e-_AMS_PRL2023_daily.csv', '/home/david/thesis/data/raw/ams/AMS_timedependent_datasets/He_AMS_PRL2025_BR.csv', '/home/david/thesis/data/raw/ams/AMS_timedependent_datasets/Li-He_AMS_PRL2025_BR.csv', '/home/david/thesis/data/raw/ams/AMS_timedependent_datasets/p_AMS_PRL2021_daily.csv', '/home/david/thesis/data/raw/ams/AMS_timedependent_datasets/6Li-7Li_AMS_PRL2025_4BR.csv', '/home/david/thesis/data/raw/ams/AMS_timedependent_datasets/C-He_AMS_PRL2025_BR.csv', '/home/david/thesis/data/raw/ams/AMS_timedepe

In [9]:
import glob

candidates = []
for p in glob.glob("/home/david/**/*.csv", recursive=True):
    low = p.lower()
    if ("grb" in low) or ("t90" in low) or ("swift" in low) or ("fermi" in low) or ("batse" in low):
        candidates.append(p)

print("candidates:", len(candidates))
print("\n".join(candidates[:50]))

candidates: 3
/home/david/thesis/outputs/tables/grb_summary_stats.csv
/home/david/thesis/data/raw/grb/fermigbrst.csv
/home/david/thesis/data/processed/grb/grb_gbm_clean.csv


In [10]:
import pandas as pd

path = "/home/david/thesis/data/processed/grb/grb_gbm_clean.csv"
df = pd.read_csv(path)

print(df.shape)
print(df.columns)
print(df.head(3))

(4208, 11)
Index(['NAME', 'RA', 'DEC', 'TRIGGER_TIME', 'T90', 'T90_ERROR', 'FLUENCE',
       'FLUENCE_ERROR', 'T50', 'T50_ERROR', 'TRIGGER_NAME'],
      dtype='str')
                             NAME        RA      DEC  TRIGGER_TIME     T90  \
0  GRB120403857                     55.3384 -89.0093  56020.856927   4.288   
1  GRB120227725                    256.7300 -88.8600  55984.725475  17.408   
2  GRB230524357                     13.0800 -87.7700  60088.357309  11.008   

   T90_ERROR       FLUENCE  FLUENCE_ERROR    T50  T50_ERROR   TRIGGER_NAME  
0      1.935  2.396400e-07   2.045800e-08  1.408      1.620  bn120403857\n  
1      0.810  2.194900e-05   1.040300e-07  6.656      0.572  bn120227725\n  
2      4.720  1.420100e-06   4.245700e-08  3.584      0.572  bn230524357\n  


In [11]:
import numpy as np
import pandas as pd

# try common T90 column names
for col in ["T90", "t90", "t90_s", "T90_s", "T90 (s)", "t90_sec", "T90_sec"]:
    if col in df.columns:
        t90_col = col
        break
else:
    # fallback: find anything containing 't90'
    t90_like = [c for c in df.columns if "t90" in c.lower()]
    raise KeyError(f"No obvious T90 column. Columns containing 't90': {t90_like}")

t90 = pd.to_numeric(df[t90_col], errors="coerce").dropna().to_numpy()

thresholds = [1, 2, 3]
N = len(t90)
print("Using column:", t90_col)
print("N =", N)

for thr in thresholds:
    k = np.sum(t90 <= thr)
    print(f"{thr}s: {k}/{N}  frac={k/N:.3f}")

Using column: T90
N = 4208
1s: 520/4208  frac=0.124
2s: 703/4208  frac=0.167
3s: 819/4208  frac=0.195


In [12]:
import numpy as np
import pandas as pd

def amp_maxmin(flux):
    fmax = np.nanmax(flux)
    fmin = np.nanmin(flux)
    return (fmax - fmin) / (fmax + fmin)

def amp_over_mean(flux):
    fmax = np.nanmax(flux)
    fmin = np.nanmin(flux)
    fmean = np.nanmean(flux)
    return (fmax - fmin) / fmean

def amp_percentile(flux, lo=5, hi=95):
    f_lo = np.nanpercentile(flux, lo)
    f_hi = np.nanpercentile(flux, hi)
    return (f_hi - f_lo) / (f_hi + f_lo)

In [13]:
def load_flux(path):
    df = pd.read_csv(path)

    # pick a likely flux column
    candidates = [c for c in df.columns if any(k in c.lower() for k in ["flux", "proton", "value"])]
    # If it's a 2-column file (time, flux), just take the 2nd column
    if len(df.columns) == 2:
        flux_col = df.columns[1]
    elif "flux" in [c.lower() for c in df.columns]:
        flux_col = [c for c in df.columns if c.lower() == "flux"][0]
    elif candidates:
        flux_col = candidates[0]
    else:
        # fallback: first numeric-looking column
        flux_col = df.select_dtypes(include="number").columns[0]

    flux = pd.to_numeric(df[flux_col], errors="coerce").to_numpy()
    return df, flux_col, flux

In [15]:
import glob

proton = [p for p in glob.glob("/home/david/thesis/data/raw/cr/AMS_timedependent_datasets/*.csv")
          if "/p_" in p or "proton" in p.lower() or p.split("/")[-1].startswith("p")]

# show ones that include those numbers in the filename
for p in proton:
    name = p.lower()
    if ("3.29" in name and "3.64" in name) or ("3.64" in name and "4.02" in name):
        print(p)

In [16]:
import glob

base = "/home/david/thesis/data/raw/cr/AMS_timedependent_datasets"
all_csv = glob.glob(base + "/*.csv")
print("CSV count:", len(all_csv))
print("\n".join(all_csv[:30]))

CSV count: 14
/home/david/thesis/data/raw/cr/AMS_timedependent_datasets/e+_AMS_PRL2023_daily.csv
/home/david/thesis/data/raw/cr/AMS_timedependent_datasets/N-He_AMS_PRL2025_BR.csv
/home/david/thesis/data/raw/cr/AMS_timedependent_datasets/e-_AMS_PRL2023_daily.csv
/home/david/thesis/data/raw/cr/AMS_timedependent_datasets/He_AMS_PRL2025_BR.csv
/home/david/thesis/data/raw/cr/AMS_timedependent_datasets/Li-He_AMS_PRL2025_BR.csv
/home/david/thesis/data/raw/cr/AMS_timedependent_datasets/p_AMS_PRL2021_daily.csv
/home/david/thesis/data/raw/cr/AMS_timedependent_datasets/6Li-7Li_AMS_PRL2025_4BR.csv
/home/david/thesis/data/raw/cr/AMS_timedependent_datasets/C-He_AMS_PRL2025_BR.csv
/home/david/thesis/data/raw/cr/AMS_timedependent_datasets/B-He_AMS_PRL2025_BR.csv
/home/david/thesis/data/raw/cr/AMS_timedependent_datasets/pbar_AMS_PRL2025_BR.csv
/home/david/thesis/data/raw/cr/AMS_timedependent_datasets/O-He_AMS_PRL2025_BR.csv
/home/david/thesis/data/raw/cr/AMS_timedependent_datasets/He_AMS_PRL2022_daily.

In [17]:
import pandas as pd

path = "/home/david/thesis/data/raw/cr/AMS_timedependent_datasets/p_AMS_PRL2021_daily.csv"
dfp = pd.read_csv(path)

print(dfp.shape)
print(dfp.columns[:50])
print(dfp.head(3))

(83757, 7)
Index(['date YYYY-MM-DD', 'rigidity_min GV', 'rigidity_max GV',
       'proton_flux m^-2sr^-1s^-1GV^-1',
       'proton_flux_error_statistical m^-2sr^-1s^-1GV^-1',
       'proton_flux_error_timedependent m^-2sr^-1s^-1GV^-1',
       'proton_flux_error_systematic_total m^-2sr^-1s^-1GV^-1'],
      dtype='str')
  date YYYY-MM-DD  rigidity_min GV  rigidity_max GV  \
0      2011-05-20             1.00             1.16   
1      2011-05-20             1.16             1.33   
2      2011-05-20             1.33             1.51   

   proton_flux m^-2sr^-1s^-1GV^-1  \
0                           999.8   
1                           974.9   
2                           914.4   

   proton_flux_error_statistical m^-2sr^-1s^-1GV^-1  \
0                                              15.7   
1                                               7.5   
2                                               6.7   

   proton_flux_error_timedependent m^-2sr^-1s^-1GV^-1  \
0                               

In [19]:
# show any columns containing "3.29" or "4.02" or "rig" or "gv"
for c in cols:
    lc = c.lower()
    if any(x in lc for x in ["3.29", "3.64", "4.02", "rig", "gv"]):
        print(c)

rigidity_min GV
rigidity_max GV
proton_flux m^-2sr^-1s^-1GV^-1
proton_flux_error_statistical m^-2sr^-1s^-1GV^-1
proton_flux_error_timedependent m^-2sr^-1s^-1GV^-1
proton_flux_error_systematic_total m^-2sr^-1s^-1GV^-1


In [21]:
import numpy as np
import pandas as pd

# column names (as in your dfp.columns)
col_rmin = "rigidity_min GV"
col_rmax = "rigidity_max GV"
col_flux = "proton_flux m^-2sr^-1s^-1GV^-1"

def fractional_amplitude(flux_series):
    x = pd.to_numeric(flux_series, errors="coerce").dropna().to_numpy()
    fmax = np.max(x)
    fmin = np.min(x)
    A = (fmax - fmin) / (fmax + fmin)
    return A, len(x), fmin, fmax

bins = [(3.29, 3.64), (3.64, 4.02)]

for rmin, rmax in bins:
    mask = np.isclose(dfp[col_rmin], rmin) & np.isclose(dfp[col_rmax], rmax)
    sub = dfp.loc[mask, [col_flux, "date YYYY-MM-DD"]].copy()

    A, N, fmin, fmax = fractional_amplitude(sub[col_flux])

    print(f"\nRigidity bin {rmin:.2f}–{rmax:.2f} GV")
    print("N =", N)
    print("A =", round(A, 3))
    print("min flux =", fmin, "max flux =", fmax)


Rigidity bin 3.29–3.64 GV
N = 2824
A = 0.326
min flux = 170.3 max flux = 334.7

Rigidity bin 3.64–4.02 GV
N = 2824
A = 0.295
min flux = 146.5 max flux = 269.2


In [22]:
import numpy as np
import pandas as pd

col_rmin = "rigidity_min GV"
col_rmax = "rigidity_max GV"
col_flux = "proton_flux m^-2sr^-1s^-1GV^-1"

def get_flux(dfp, rmin, rmax):
    m = np.isclose(dfp[col_rmin], rmin) & np.isclose(dfp[col_rmax], rmax)
    x = pd.to_numeric(dfp.loc[m, col_flux], errors="coerce").dropna().to_numpy()
    return x

def stats_for(x):
    fmin, fmax = np.min(x), np.max(x)
    f5, f95 = np.percentile(x, [5, 95])
    f10, f90 = np.percentile(x, [10, 90])
    return {
        "A_maxmin_(max-min)/(max+min)": (fmax - fmin) / (fmax + fmin),
        "ln(max/min)": np.log(fmax / fmin),
        "ln(p95/p5)": np.log(f95 / f5),
        "ln(p90/p10)": np.log(f90 / f10),
        "p95-p5 over (p95+p5)": (f95 - f5) / (f95 + f5),
    }

bins = [(3.29, 3.64), (3.64, 4.02)]

for rmin, rmax in bins:
    x = get_flux(dfp, rmin, rmax)
    s = stats_for(x)
    print(f"\nBin {rmin:.2f}–{rmax:.2f} GV   N={len(x)}")
    for k, v in s.items():
        print(f"{k:32s} = {v:.3f}")


Bin 3.29–3.64 GV   N=2824
A_maxmin_(max-min)/(max+min)     = 0.326
ln(max/min)                      = 0.676
ln(p95/p5)                       = 0.455
ln(p90/p10)                      = 0.411
p95-p5 over (p95+p5)             = 0.224

Bin 3.64–4.02 GV   N=2824
A_maxmin_(max-min)/(max+min)     = 0.295
ln(max/min)                      = 0.608
ln(p95/p5)                       = 0.404
ln(p90/p10)                      = 0.363
p95-p5 over (p95+p5)             = 0.199


In [24]:
import numpy as np
import pandas as pd

path_p = "/home/david/thesis/data/raw/cr/AMS_timedependent_datasets/p_AMS_PRL2021_daily.csv"
dfp = pd.read_csv(path_p)

col_rmin = "rigidity_min GV"
col_rmax = "rigidity_max GV"
col_flux = "proton_flux m^-2sr^-1s^-1GV^-1"

def A_maxmin(series):
    x = pd.to_numeric(series, errors="coerce").dropna().to_numpy()
    return (np.max(x) - np.min(x)) / (np.max(x) + np.min(x)), len(x)

m = np.isclose(dfp[col_rmin], 3.29) & np.isclose(dfp[col_rmax], 3.64)
A_p, N_p = A_maxmin(dfp.loc[m, col_flux])

print("AMS-02 protons (3.29–3.64)  N =", N_p, " A =", round(A_p, 3))

AMS-02 protons (3.29–3.64)  N = 2824  A = 0.326


In [25]:
import glob, os

hits = []
for p in glob.glob("/home/david/thesis/**/*.csv", recursive=True):
    name = os.path.basename(p).lower()
    if any(k in name for k in ["agn", "bl", "bll", "fsrq", "variab", "fractional", "fvar", "4lac"]):
        hits.append(p)

print("candidates:", len(hits))
print("\n".join(hits[:50]))

candidates: 2
/home/david/thesis/outputs/tables/agn_summary_stats.csv
/home/david/thesis/data/processed/agn/agn_4lac_clean.csv


In [27]:
import pandas as pd

agn_path = "/home/david/thesis/data/processed/agn/agn_4lac_clean.csv"
dfa = pd.read_csv(agn_path)

print(dfa.shape)
print(dfa.columns)
print(dfa.head(5))

(3407, 18)
Index(['CLASS', 'DEC_Counterpart', 'Energy_Flux100', 'Flux1000',
       'Frac_Variability', 'LP_Index', 'PL_Index', 'RAJ2000', 'RA_Counterpart',
       'Redshift', 'SED_class', 'Source_Name', 'Unc_Energy_Flux100',
       'Unc_Flux1000', 'Unc_Frac_Variability', 'Unc_LP_Index', 'Unc_PL_Index',
       'Variability_Index'],
      dtype='str')
     CLASS  DEC_Counterpart  Energy_Flux100      Flux1000  Frac_Variability  \
0   b'bcu'        47.700201    1.499454e-12  1.259796e-10          0.675882   
1   b'bll'        -7.774145    8.339171e-12  7.471219e-10          0.406565   
2   b'bll'        -0.194420    1.231385e-12  1.082246e-10          0.000000   
3  b'fsrq'        21.226743    2.555889e-11  1.347354e-09          0.996138   
4   b'bcu'       -41.923705    3.560476e-12  2.821817e-10          0.490977   

   LP_Index  PL_Index  RAJ2000  RA_Counterpart  Redshift SED_class  \
0  2.254081  2.271696   0.3126        0.329341      -inf    b'ISP'   
1  2.078927  2.116692   0.3151   

In [28]:
import numpy as np
import pandas as pd

# Find likely class column
class_candidates = [c for c in dfa.columns if any(k in c.lower() for k in ["class", "type", "source", "agn"])]
print("class candidates:", class_candidates)

# Find likely fractional variability column
fvar_candidates = [c for c in dfa.columns if any(k in c.lower() for k in ["fvar", "frac", "variab"])]
print("fvar candidates:", fvar_candidates)

class candidates: ['CLASS', 'SED_class', 'Source_Name']
fvar candidates: ['Frac_Variability', 'Unc_Frac_Variability', 'Variability_Index']


In [29]:
import numpy as np
import pandas as pd

A_proton = 0.326  # from your AMS proton calculation

class_col = class_candidates[0]   # <-- replace if needed
fvar_col  = fvar_candidates[0]    # <-- replace if needed

cls = dfa[class_col].astype(str).str.lower()
fvar = pd.to_numeric(dfa[fvar_col], errors="coerce")

bl_mask   = cls.str.contains("bll") | cls.str.contains("bl lac") | cls.str.contains("bllac")
fsrq_mask = cls.str.contains("fsrq")

bl_med = np.nanmedian(fvar[bl_mask])
fsrq_med = np.nanmedian(fvar[fsrq_mask])
diff = fsrq_med - bl_med

print("BL Lacs median frac variability =", round(bl_med, 3), " ratio =", round(bl_med/A_proton, 3))
print("FSRQs median frac variability   =", round(fsrq_med, 3), " ratio =", round(fsrq_med/A_proton, 3))
print("FSRQ - BL Lac difference        =", round(diff, 3), " ratio =", round(diff/A_proton, 3))
print("AMS-02 proton amplitude A       =", round(A_proton, 3), " ratio =", round(A_proton/A_proton, 3))

BL Lacs median frac variability = 0.279  ratio = 0.857
FSRQs median frac variability   = 0.649  ratio = 1.99
FSRQ - BL Lac difference        = 0.369  ratio = 1.132
AMS-02 proton amplitude A       = 0.326  ratio = 1.0


In [2]:
import numpy as np
import pandas as pd

# Load cleaned GBM catalog
path = "/home/david/thesis/data/processed/grb/grb_gbm_clean.csv"
df = pd.read_csv(path)

# Detect a T90 column
preferred = ["T90", "t90", "t90_s", "T90_s", "T90 (s)", "t90_sec", "T90_sec"]
t90_col = next((c for c in preferred if c in df.columns), None)

if t90_col is None:
    candidates = [c for c in df.columns if "t90" in c.lower()]
    if not candidates:
        raise KeyError("No T90-like column found. Columns:\n" + "\n".join(df.columns))
    # pick first candidate that has numeric values
    for c in candidates:
        if pd.to_numeric(df[c], errors="coerce").notna().any():
            t90_col = c
            break
    if t90_col is None:
        raise KeyError(f"T90-like columns found but none numeric: {candidates}")

# Clean T90 values (seconds)
t90 = pd.to_numeric(df[t90_col], errors="coerce").dropna().to_numpy()
t90 = t90[np.isfinite(t90)]
t90 = t90[t90 > 0]

# Compute stats
N = len(t90)
n_short = int(np.sum(t90 < 2.0))
n_long = int(np.sum(t90 >= 2.0))
short_frac = n_short / N if N else np.nan
med = float(np.median(t90)) if N else np.nan
p16, p84 = (np.percentile(t90, [16, 84]) if N else (np.nan, np.nan))

# Print results (no table wording)
print("Using T90 column:", t90_col)
print("N =", N)
print("N_short (T90 < 2 s) =", n_short)
print("N_long  (T90 ≥ 2 s) =", n_long)
print("short_fraction =", f"{short_frac:.3f}")
print("median_T90_s =", f"{med:.3f}")
print("T90_16_84_percentile_s =", f"{p16:.3f}", "to", f"{p84:.3f}")

Using T90 column: T90
N = 4208
N_short (T90 < 2 s) = 703
N_long  (T90 ≥ 2 s) = 3505
short_fraction = 0.167
median_T90_s = 18.688
T90_16_84_percentile_s = 1.792 to 64.001


In [3]:
import numpy as np
import pandas as pd

agn_path = "/home/david/thesis/data/processed/agn/agn_4lac_clean.csv"
dfa = pd.read_csv(agn_path)

print("shape:", dfa.shape)
print("columns:", list(dfa.columns))

# --- choose columns (auto-detect with fallbacks) ---
# class column
class_candidates = [c for c in dfa.columns if any(k in c.lower() for k in ["class", "type", "subclass", "source_class"])]
# photon index column (PL index)
pl_candidates = [c for c in dfa.columns if any(k in c.lower() for k in ["pl_index", "photon", "photon_index", "plindex", "spectral_index", "index"])]

# fractional variability column
fvar_candidates = [c for c in dfa.columns if any(k in c.lower() for k in ["frac_variability", "frac", "variab", "fvar"])]

print("class_candidates:", class_candidates)
print("pl_candidates:", pl_candidates)
print("fvar_candidates:", fvar_candidates)

shape: (3407, 18)
columns: ['CLASS', 'DEC_Counterpart', 'Energy_Flux100', 'Flux1000', 'Frac_Variability', 'LP_Index', 'PL_Index', 'RAJ2000', 'RA_Counterpart', 'Redshift', 'SED_class', 'Source_Name', 'Unc_Energy_Flux100', 'Unc_Flux1000', 'Unc_Frac_Variability', 'Unc_LP_Index', 'Unc_PL_Index', 'Variability_Index']
class_candidates: ['CLASS', 'SED_class']
pl_candidates: ['LP_Index', 'PL_Index', 'Unc_LP_Index', 'Unc_PL_Index', 'Variability_Index']
fvar_candidates: ['Frac_Variability', 'Unc_Frac_Variability', 'Variability_Index']


In [8]:
# Example strict mask: only finite values and (optionally) a SNR-like cut if error column exists
strict_mask = np.isfinite(pd.to_numeric(dfa[pl_col], errors="coerce")) & np.isfinite(pd.to_numeric(dfa[fvar_col], errors="coerce"))

# OPTIONAL: if you have an error column for fractional variability, use it:
err_candidates = [c for c in dfa.columns if ("err" in c.lower() or "error" in c.lower()) and ("var" in c.lower() or "fvar" in c.lower())]
print("possible fvar error cols:", err_candidates)

# If you identify an error column, uncomment and set:
# fvar_err_col = err_candidates[0]
# strict_mask = strict_mask & (pd.to_numeric(dfa[fvar_col], errors="coerce") / pd.to_numeric(dfa[fvar_err_col], errors="coerce") >= 3)

strict = subclass_medians(dfa, class_col, pl_col, fvar_col, extra_mask=strict_mask)
strict

possible fvar error cols: []


{'Gamma_BLL': 1.9357148,
 'Gamma_FSRQ': 2.38066,
 'Fvar_BLL': 0.27943447,
 'Fvar_FSRQ': 0.6485875,
 'N_BLL': 1379,
 'N_FSRQ': 755}

In [13]:
print("BLL total:", bll.sum(), "FSRQ total:", fsrq.sum())
print("BLL PL notna:", (bll & pl.notna()).sum(), "BLL Fvar notna:", (bll & fv.notna()).sum(), "BLL both:", (bll & pl.notna() & fv.notna()).sum())
print("FSRQ PL notna:", (fsrq & pl.notna()).sum(), "FSRQ Fvar notna:", (fsrq & fv.notna()).sum(), "FSRQ both:", (fsrq & pl.notna() & fv.notna()).sum())

BLL total: 1379 FSRQ total: 755
BLL PL notna: 1379 BLL Fvar notna: 1379 BLL both: 1379
FSRQ PL notna: 755 FSRQ Fvar notna: 755 FSRQ both: 755


In [14]:
[c for c in dfa.columns if any(k in c.lower() for k in ["err", "error", "unc", "ts", "signif", "flag", "npts", "bins", "lc"])]

['Unc_Energy_Flux100',
 'Unc_Flux1000',
 'Unc_Frac_Variability',
 'Unc_LP_Index',
 'Unc_PL_Index']

In [15]:
import pandas as pd
p = "/home/david/thesis/outputs/tables/agn_summary_stats.csv"
d = pd.read_csv(p)
print(d.columns)
print(d.head(20))

Index(['N_total_sources', 'N_BLL', 'N_FSRQ', 'Gamma_median_BLL',
       'Gamma_median_FSRQ', 'Variability_median_BLL',
       'Variability_median_FSRQ'],
      dtype='str')
   N_total_sources  N_BLL  N_FSRQ  Gamma_median_BLL  Gamma_median_FSRQ  \
0             3407   1379     755          2.023715           2.449434   

   Variability_median_BLL  Variability_median_FSRQ  
0                0.279434                 0.648587  


In [16]:
import numpy as np
import pandas as pd

dfa = pd.read_csv("/home/david/thesis/data/processed/agn/agn_4lac_clean.csv")

# pick the class column you used before
cls = dfa[class_col].astype(str).str.lower()
bll  = cls.str.contains("bll") | cls.str.contains("bl lac") | cls.str.contains("bllac")
fsrq = cls.str.contains("fsrq")

# candidate "index" columns
cand = [c for c in dfa.columns if "index" in c.lower() or "photon" in c.lower() or "pl_" in c.lower()]
print("candidates:", cand)

def med(x, m):
    x = pd.to_numeric(x, errors="coerce")
    return float(np.nanmedian(x[m]))

for c in cand:
    mb = med(dfa[c], bll)
    mf = med(dfa[c], fsrq)
    if np.isfinite(mb) and np.isfinite(mf):
        print(f"{c:35s}  BLL={mb:.4f}  FSRQ={mf:.4f}")

candidates: ['LP_Index', 'PL_Index', 'Unc_LP_Index', 'Unc_PL_Index', 'Variability_Index']
LP_Index                             BLL=1.9357  FSRQ=2.3807
PL_Index                             BLL=2.0237  FSRQ=2.4494
Unc_LP_Index                         BLL=0.1171  FSRQ=0.0924
Unc_PL_Index                         BLL=0.0853  FSRQ=0.0664
Variability_Index                    BLL=18.4986  FSRQ=76.7468


In [18]:
import pandas as pd

def dataset_fingerprint(path, n=3):
    df = pd.read_csv(path)
    print("PATH:", path)
    print("SHAPE:", df.shape)
    print("COLUMNS:", list(df.columns))
    print("HEAD:")
    print(df.head(n))
    return df

df_grb = dataset_fingerprint("/home/david/thesis/data/processed/grb/grb_gbm_clean.csv")
dfa    = dataset_fingerprint("/home/david/thesis/data/processed/agn/agn_4lac_clean.csv")
dfp    = dataset_fingerprint("/home/david/thesis/data/raw/cr/AMS_timedependent_datasets/p_AMS_PRL2021_daily.csv")

PATH: /home/david/thesis/data/processed/grb/grb_gbm_clean.csv
SHAPE: (4208, 11)
COLUMNS: ['NAME', 'RA', 'DEC', 'TRIGGER_TIME', 'T90', 'T90_ERROR', 'FLUENCE', 'FLUENCE_ERROR', 'T50', 'T50_ERROR', 'TRIGGER_NAME']
HEAD:
                             NAME        RA      DEC  TRIGGER_TIME     T90  \
0  GRB120403857                     55.3384 -89.0093  56020.856927   4.288   
1  GRB120227725                    256.7300 -88.8600  55984.725475  17.408   
2  GRB230524357                     13.0800 -87.7700  60088.357309  11.008   

   T90_ERROR       FLUENCE  FLUENCE_ERROR    T50  T50_ERROR   TRIGGER_NAME  
0      1.935  2.396400e-07   2.045800e-08  1.408      1.620  bn120403857\n  
1      0.810  2.194900e-05   1.040300e-07  6.656      0.572  bn120227725\n  
2      4.720  1.420100e-06   4.245700e-08  3.584      0.572  bn230524357\n  
PATH: /home/david/thesis/data/processed/agn/agn_4lac_clean.csv
SHAPE: (3407, 18)
COLUMNS: ['CLASS', 'DEC_Counterpart', 'Energy_Flux100', 'Flux1000', 'Frac_Variabi

In [19]:
CONFIG = {
    "GRB": {"short_threshold_s": 2.0},
    "AMS": {"rigidity_bin": (3.29, 3.64), "A_def": "(max-min)/(max+min)"},
    "AGN": {"index_col": "PL_Index", "var_col": "Frac_Variability"},
}
CONFIG

{'GRB': {'short_threshold_s': 2.0},
 'AMS': {'rigidity_bin': (3.29, 3.64), 'A_def': '(max-min)/(max+min)'},
 'AGN': {'index_col': 'PL_Index', 'var_col': 'Frac_Variability'}}

In [20]:
assert n_short + n_long == N
assert abs((0.279/0.326) - 0.857) < 0.01

In [21]:
import numpy as np
import pandas as pd

t90 = pd.to_numeric(df_grb["T90"], errors="coerce").dropna().to_numpy()
t90 = t90[np.isfinite(t90)]
t90 = t90[t90 > 0]

thr = CONFIG["GRB"]["short_threshold_s"]

N = len(t90)
n_short = int((t90 < thr).sum())
n_long  = int((t90 >= thr).sum())
short_frac = n_short / N

med = float(np.median(t90))
p16, p84 = np.percentile(t90, [16, 84])

grb_stats = {
    "N": N,
    f"N_short_T90<{thr}s": n_short,
    f"N_long_T90>={thr}s": n_long,
    "short_fraction": round(short_frac, 3),
    "median_T90_s": round(med, 3),
    "T90_p16_s": round(float(p16), 3),
    "T90_p84_s": round(float(p84), 3),
}

grb_stats

{'N': 4208,
 'N_short_T90<2.0s': 703,
 'N_long_T90>=2.0s': 3505,
 'short_fraction': 0.167,
 'median_T90_s': 18.688,
 'T90_p16_s': 1.792,
 'T90_p84_s': 64.001}

In [22]:
out_grb = pd.DataFrame([grb_stats])
out_grb.to_csv("/home/david/thesis/outputs/tables/grb_summary_recomputed.csv", index=False)
out_grb

,N,N_short_T90<2.0s,N_long_T90>=2.0s,short_fraction,median_T90_s,T90_p16_s,T90_p84_s
0,4208,703,3505,0.167,18.688,1.792,64.001


In [23]:
import numpy as np
import pandas as pd

col_rmin = "rigidity_min GV"
col_rmax = "rigidity_max GV"
col_flux = "proton_flux m^-2sr^-1s^-1GV^-1"

def A_maxmin(flux_series):
    x = pd.to_numeric(flux_series, errors="coerce").dropna().to_numpy()
    x = x[np.isfinite(x)]
    fmax, fmin = np.max(x), np.min(x)
    return (fmax - fmin) / (fmax + fmin), len(x), fmin, fmax

rmin, rmax = CONFIG["AMS"]["rigidity_bin"]
mask = np.isclose(dfp[col_rmin], rmin) & np.isclose(dfp[col_rmax], rmax)

A_p, N_p, fmin, fmax = A_maxmin(dfp.loc[mask, col_flux])

ams_stats = {
    "rigidity_bin_GV": f"{rmin:.2f}-{rmax:.2f}",
    "N": N_p,
    "A": round(float(A_p), 3),
    "min_flux": float(fmin),
    "max_flux": float(fmax),
}
ams_stats

{'rigidity_bin_GV': '3.29-3.64',
 'N': 2824,
 'A': 0.326,
 'min_flux': 170.3,
 'max_flux': 334.7}

In [24]:
pd.DataFrame([ams_stats]).to_csv("/home/david/thesis/outputs/tables/ams_proton_A_recomputed.csv", index=False)

In [25]:
import numpy as np
import pandas as pd

class_col = "CLASS"
pl_col = CONFIG["AGN"]["index_col"]        # "PL_Index"
fvar_col = CONFIG["AGN"]["var_col"]        # "Frac_Variability"

cls = dfa[class_col].astype(str).str.lower()
bll  = cls.str.contains("bll") | cls.str.contains("bl lac") | cls.str.contains("bllac")
fsrq = cls.str.contains("fsrq")

pl = pd.to_numeric(dfa[pl_col], errors="coerce")
fv = pd.to_numeric(dfa[fvar_col], errors="coerce")

Gamma_BLL  = float(np.nanmedian(pl[bll]))
Gamma_FSRQ = float(np.nanmedian(pl[fsrq]))
Fvar_BLL   = float(np.nanmedian(fv[bll]))
Fvar_FSRQ  = float(np.nanmedian(fv[fsrq]))

A = float(A_p)

agn_stats = {
    "N_total_sources": int(len(dfa)),
    "N_BLL": int(bll.sum()),
    "N_FSRQ": int(fsrq.sum()),
    "Gamma_median_BLL": round(Gamma_BLL, 4),
    "Gamma_median_FSRQ": round(Gamma_FSRQ, 4),
    "Variability_median_BLL": round(Fvar_BLL, 6),
    "Variability_median_FSRQ": round(Fvar_FSRQ, 6),
    "Variability_ratio_BLL_to_A": round(Fvar_BLL / A, 3),
    "Variability_ratio_FSRQ_to_A": round(Fvar_FSRQ / A, 3),
    "Variability_ratio_diff_to_A": round((Fvar_FSRQ - Fvar_BLL) / A, 3),
}
agn_stats

{'N_total_sources': 3407,
 'N_BLL': 1379,
 'N_FSRQ': 755,
 'Gamma_median_BLL': 2.0237,
 'Gamma_median_FSRQ': 2.4494,
 'Variability_median_BLL': 0.279434,
 'Variability_median_FSRQ': 0.648587,
 'Variability_ratio_BLL_to_A': 0.858,
 'Variability_ratio_FSRQ_to_A': 1.992,
 'Variability_ratio_diff_to_A': 1.134}

In [26]:
pd.DataFrame([agn_stats]).to_csv("/home/david/thesis/outputs/tables/agn_summary_recomputed.csv", index=False)

In [27]:
all_results = {
    "GRB": grb_stats,
    "AMS": ams_stats,
    "AGN": agn_stats,
}
all_results

{'GRB': {'N': 4208,
  'N_short_T90<2.0s': 703,
  'N_long_T90>=2.0s': 3505,
  'short_fraction': 0.167,
  'median_T90_s': 18.688,
  'T90_p16_s': 1.792,
  'T90_p84_s': 64.001},
 'AMS': {'rigidity_bin_GV': '3.29-3.64',
  'N': 2824,
  'A': 0.326,
  'min_flux': 170.3,
  'max_flux': 334.7},
 'AGN': {'N_total_sources': 3407,
  'N_BLL': 1379,
  'N_FSRQ': 755,
  'Gamma_median_BLL': 2.0237,
  'Gamma_median_FSRQ': 2.4494,
  'Variability_median_BLL': 0.279434,
  'Variability_median_FSRQ': 0.648587,
  'Variability_ratio_BLL_to_A': 0.858,
  'Variability_ratio_FSRQ_to_A': 1.992,
  'Variability_ratio_diff_to_A': 1.134}}

In [28]:
import numpy as np
import pandas as pd

col_rmin = "rigidity_min GV"
col_rmax = "rigidity_max GV"
col_flux = "proton_flux m^-2sr^-1s^-1GV^-1"

def A_maxmin_over_mean(flux_series):
    x = pd.to_numeric(flux_series, errors="coerce").dropna().to_numpy()
    x = x[np.isfinite(x)]
    jmax, jmin = np.max(x), np.min(x)
    jmean = np.mean(x)
    A = (jmax - jmin) / jmean
    return A, len(x), jmin, jmax, jmean

bins = [(33.50, 48.50), (48.50, 69.70), (69.70, 100.00)]

rows = []
for rmin, rmax in bins:
    m = np.isclose(dfp[col_rmin], rmin) & np.isclose(dfp[col_rmax], rmax)
    A, N, jmin, jmax, jmean = A_maxmin_over_mean(dfp.loc[m, col_flux])
    rows.append({
        "Rigidity bin (GV)": f"{rmin:.2f}–{rmax:.2f}",
        "N": N,
        "A=(Jmax-Jmin)/<J>": A,
        "Jmin": jmin,
        "Jmax": jmax,
        "<J>": jmean
    })

ams_rigidity_var = pd.DataFrame(rows)
ams_rigidity_var

,Rigidity bin (GV),N,A=(Jmax-Jmin)/<J>,Jmin,Jmax,<J>
0,33.50–48.50,2824,0.087324,0.55110,0.6018,0.580595
1,48.50–69.70,2824,0.081025,0.19980,0.2166,0.207344
2,69.70–100.00,2824,0.090067,0.07211,0.0789,0.075388
